# Frequently Asked Questions
This notebook contains a list of frequently asked questions and their answers.  If the answer is short, example code is given directly in this notebook; if not, then the reader is referred to other pyGSTi tutorials and examples.

## Contents
- [What is GST?](#what_is_gst)
- [What is pyGSTi?](#what_is_pygsti)
- [How do I run GST (1 or 2 qubits)?](#how_do_I_run_GST)
- [How do I constrain my gates to be Trace-Preserving (TP)?](#how_constrain_TP)
- [How do I constrain my gates to be CPTP?](#how_constrain_CPTP)
- [What is gauge optimization?](#what_is_gaugeopt)
- [How do I specify the details of gauge optimization?](#how_specify_gaugeopt)
- [2-qubit GST is running very slowly - how do I speed it up?](#twoQ_GST_is_slow)
- [Generating reports takes a long time - can it go any faster?](#reportGen_is_slow)


---

<a id="what_is_gst"></a>

## What is GST?
"Gate Set Tomography" (GST) is a protocol that attempts to infer the quantum operations a device is performing by analyzing  data generated by it.  The device is viewed as a black box, so that physics-level details of its construction and operation are not needed.  GST characterizes quantum gate operations, state preparations, and measurement operations simultaneously, avoiding the chicken-and-egg problem of state and process tomography, which has been shown to be problematic.  For more information about the algorithms and theory of GST, please see [*Demonstration of qubit operations below a rigorous fault tolerance threshold with gate set tomography*, Nature Communications **8** (2017)](http://dx.doi.org/10.1038/ncomms14485)

<a id="what_is_pygsti"></a>

## What is pyGSTi?
PyGSTi is the name of (this) software, which implements the various parts of the GST protocol(s).  It is written entirely in Python, and is released as an open-source software under the GPL license.  "pyGSTi" stands for "python GST implementation".

<a id="how_do_I_run_GST"></a>

## How do I run GST (1 or 2 qubits)?
Using pyGSTi, running GST is fairly simple.  Single-qubit GST is demonstrated in full in the [first tutorial](Tutorials/00 Getting Started.ipynb), and 2-qubit GST (though very similar to 1-qubit GST) is separately demonstrated in [this 2Q-GST example](Examples/2Q-GST: running it.ipynb).  The core GST algorithm is run by calling `pygsti.do_long_sequence_gst` or `pygsti.do_stdpractice_gst`.

<a id="how_constrain_TP"></a>

## How do I constrain my gates to be Trace-Preserving (TP)?
Constraint on the gates (and SPAM operations) in pyGSTi are imposed via the "parameterization" of `GateSet` objects.  Each `GateSet` essentially knows how to map some vector of (real-valued) "parameters" to a full set of gate matrices and SPAM vectors - and whether a given set of matrices & vectors can be reached depends on whether there exists a set of parameters that maps onto it.  The parameterization used by a `GateSet` object can be changed among commonly used parameterizations via the `set_all_parameterizations` member function.  To constrain gates and SPAM operations to be trace preserving, simply call





In [ ]:
my_target_gateset.set_all_parameterizations("TP")

before calling `do_long_sequence_gst` or `do_stdpractice_gst` with `my_target_gateset`.

<a id="how_constrain_CPTP"></a>

## How do I constrain my gates to be CPTP?
Similar to the above question, you simply set the parameterization of your target `GateSet` to "CPTP" before calling `do_long_sequence_gst` or `do_stdpractice_gst`.  Note that currenlty this *only ensures that the gates are CPTP* - not the SPAM operations.  (Attempts to parameterize the SPAM operations as such have resulted in poor performance and are considered an experimental feature.)

In [ ]:
my_target_gateset.set_all_parameterizations("CPTP")

<a id="what_is_gaugeopt"></a>

## What is gauge optimization?
A "gauge transformation" is a special type of mapping from one gate set to another which preserves all predicted physical outcomes (hence the name "gauge").  Thus, gauge transformations map between physically equivalent gate sets, and the set of all gauge transformations partitions the space of gate sets into physically-distinct equivalence classes.  **Gauge optimization** refers to the process by which one finds the gate set within an equivalence class which optimizes some metric - typically the frobenius distance between the gate set and some "target gate set".  In pyGSTi, gauge optimization is invoked using the `pygsti.gaugeopt_to_target` method (which minimizes the distance between a `GateSet` and a target `GateSet`) or the `pygsti.gaugeopt_custom` method (which accepts a user-defined objective function).

<a id="how_specify_gaugeopt"></a>

## How do I specify the details of gauge optimization?
Specifying the "details" of gauge optimization is essentially specifying the objective function.  The arguments of `gaugeopt_to_target` (detailed in its docstring and [pyGSTi's online docs](http://pygsti.readthedocs.io)) describe how to change which distance metric is used (the frobenius distance, process fidelity, or the Jamiolkowski trace distance) when comparing gates, and how to put different weighting factors on individual gates (via the `itemWeights` argument).    In higher-level functions, gauge-optimization details are specified by simply creating a dictionary of `gauge_opt_to_target`'s arguments.  Adding a new gauge-optimization to an existing `pygsti.objects.Results` object is demonstrated in the ["adding new gauge optimizations" example](Examples/GOpt: adding new gauge optimizations.ipynb).  One particularly powerful way to control gauge optimization is by specifying an imperfect target `GateSet` which includes the errors you expect to see in the gates.  This is simple to do, and outlined in the ["non-ideal gauge optimization targets" example](Examples/GOpt: non-ideal gauge optimization targets.ipynb).

<a id="twoQ_GST_is_slow"></a>

## 2-qubit GST is running very slowly - how do I speed it up?
There are two primary ways of speeding up the GST analysis within pyGSTi: 1) using more/multiple processors and 2) reducing the number of gate sequences being analyzed.  The first option is achieved by passing the core pyGSTi routines (e.g. `do_long_sequence_gst`) a `mpi4py.MPI.Comm` object (and often a per-core memory limit too), and is demonstrated in an [example](Examples/MPI: running GST in parallel.ipynb).  The second option involves more of a tradeoff, as decreasing the number of sequences with reduce the accuracy of GST and/or its robustness.  Restricting GST to shorter sequences (reducing the "max-lengths" used) will result in less accurate estimates, i.e. larger error bars.  The use of "fiducial pair reduction" techniques, outlined in [this tutorial](Tutorials/10 Fiducial Pair Reduction.ipynb), retains the accuracy of GST while eliminating sequences.  This, however, results is less robustness and sensitivity to out-of-model errors.

<a id="reportGen_is_slow"></a>

## Generating reports takes a long time - can it go any faster?
Currently there aren't many options for speeding up the core computation required to generate a report (using, e.g. `pygsti.report.create_general_report`.  There are two ways, however, in which the caching built into the pyGSTi `Results` and `Workspace` objects can effectively speedup report-generation times.  

**First, the computation of error bars, if requested is often the slowest part of report generation by far.**  This computation can be split into two parts: 1) computing a Hessian and 2) optimizing a "gauge metric", both of which can take considerable time (especially for 2+ qubits!).  To avoid repeating these steps, a `pygsti.objects.Results` object (actually the `pygsti.objects.Estimate` objects contained therein) will cache any computed Hessians and related error-bar quantities.  Furthermore, the Hessian computation can be performed on multiple cores.  Thus, if you know you're going to need error bars, it can useful to simply call `get_confidence_region` on an `Estimate` right after the call to `do_long_sequence_gst` and prior to generating the report - perhaps on a larger machine with more processors.  `get_confidence_region` will cache the resulting confidence region so that it won't need to be computed during the report generation.  This technique is illustrated in the [2Q-GST error bars example](Examples/2Q-GST: generating error bars.ipynb).  Somewhat independent of computing a Hessian is specifying how the Hessian is projected onto a space orthogonal to the manifold of gauge degrees of freedom.  The basic idea here is that there is a choice, similar to the choice of which metric to use for gauge optimization, which defines the notion of "orthogonality" and thereby how error bars are constructed for the non-gauge degrees of freedom.  The default in pyGSTi is to optimize this metric so as to make the error bars as small as possible - but this can sometimes take a long time.  To avoid this "metric optimization" one can set the `"hessianProjection"` parameter to either `"std"` or `"intrinsic"`.  For example:

In [ ]:
results.estimates['default'].parameters['hessianProjection'] = 'std' # or 'intrinsic'

The `"std"` option uses a fixed metric that should be decent but almost certainly isn't optimal.  The `"intrinsic"` option tries a little harder to infer what a good metric would be while avoiding any heavy computation - but its use is still experimental.  Using either is fine, but may result in artificially large error bars.  So if you use one of these alternative `"hessianProjection"` values and get error bars you're happy with, then great.  If the error bars seem too large, it's probably worth setting the parameter back to `"optimal gate CIs"`, its default value.

In [ ]:
results.estimates['default'].parameters['hessianProjection'] = 'optimal gate CIs' # the default

**Secondly, `create_general_report` returns a `Workspace` object that caches the various tables and plots within a report.**  This `Workspace` is can be stored and given as the `ws` argument to a subsequent call to `create_general_report`, and this latter call with utilize the cache within that `Workspace`.  This is useful if you want to construct two version of the same, *or similar*, reports.  For instance, you might want to generate one "full" report (with `brief=False`, the default) and one "brief" report from the same results.  Furthermore, note that specifying `None` as the `filename` argument to `create_general_report` results in not output files but *does* construct a `Workspace` full of all the would-be report's items.  This functionality can be used to generate a `Workspace` on a large and fast multi-processor machine (which ran the main GST analysis).  For example, using a workspace as shown below would dramatically speed up the 2nd and 3rd `create_general_report` calls.

In [ ]:
#On large & fast computer
results = pygsti.do_long_sequence_gst(..., comm=my_comm)
results.estimates['default'].get_confidence_region(95, comm=my_comm) #if we want 95% CI error bars
ws = pygsti.report.create_general_report(results, None, "Dummy Title") #no output produced, just to create ws
pickle.dump( (results, ws), open("my_saved_results.pkl","wb") )

#On small laptop
results, ws = pickle.load( open("my_saved_results.pkl","rb") )
pygsti.report.create_general_report(results, "full_report.html", "My Full Report", ws=ws) # FAST!
pygsti.report.create_general_report(results, "brief_report.html", "My Brief Report", ws=ws, brief=True) # FAST!